In [1]:
import numpy as np
import pandas as pd
import h5py
import os
from tqdm.notebook import tqdm
from datetime import datetime

CURR_DIR = os.getcwd()
HIS_types = ['FAULT', 'SHEAR-ZONE', 'FOLD', 'TILT', 'UNCONFORMITY', 'DYKE', 'PLUG']
PATH_TO_MODELS = 

In [3]:
# how to get my models
from methods.io_things import som_foo

som_foo()

1234

In [6]:
import random
import gzip
from urllib.request import urlopen

def display_models(his_filter, display_number):
    # download file, ungzip and stuff into numpy array
    def get_gz_array(url, skiprows):
        my_gzip_stream = urlopen(url)
        my_stream = gzip.open(my_gzip_stream, 'r')
        return (np.loadtxt(my_stream, skiprows=skiprows))

     # !!! Добавила распарсивание g00 файла
    def get_gz_array_g00(url, skiprows):
        my_gzip_stream = urlopen(url)
        my_stream = gzip.open(my_gzip_stream, 'r')
        return my_stream

    def create_rock_type_dict(my_stream):
        rock_types_id = {}
        for line in my_stream:
            if "ROCK DEFINITION" in str(line):
                rock_type = str(line).split(" ")[2][0:-3]
                rock_type_number = int(str(line).split(" ")[-1][0:-3])
                if rock_type in rock_types_id:
                    rock_types_id[rock_type].append(rock_type_number)
                else:
                    rock_types_id[rock_type] = [rock_type_number]

        return rock_types_id

    url = 'https://cloudstor.aarnet.edu.au/plus/s/8ZT6tjOvoLWmLPx/download?path=%2f'
    used = []
    z = 0
    fail = 0
    models_list = []
    models_names_list = []
    models_types_id_list = [] # добавляем идентификацию типов
    models = pd.read_csv(f'{CURR_DIR}/model_list/models.csv')

    models2 = models[
        models['event03'].str.contains(his_filter[0])
        & models['event04'].str.contains(his_filter[1])
        & models['event05'].str.contains(his_filter[2])]
    models2 = models2.reset_index(drop=True)

    model_number2 = len(models2)
    if (model_number2):
        print("sampling from", model_number2, "models matching filter", his_filter)
    else:
        print("no models found with filter", his_filter, "check list syntax and spelling of events")
        return ()

    while z < display_number and fail < 1000:
        ran = random.randint(0, model_number2 - 1)
        if (ran in used):
            continue
        else:
            used.append(ran)

        file_split = models2.iloc[ran]['root'].split('/')
        tail = models2.iloc[ran]['event03'] + '_' + models2.iloc[ran]['event04'] + '_' + models2.iloc[ran][
            'event05'] + '&files='
        root = url + tail + file_split[2]
        path = root + '.g12.gz'
        mod = get_gz_array(path, skiprows=0)
        mod2 = mod.reshape((200, 200, 200))
        mod2 = np.transpose(mod2, (1, 2, 0)) # HERE!!!!
        # print(ran, file_split[2], 'STRATIGRAPHY TILT', models2.iloc[ran]['event_all'])
        models_list.append(mod2)
        models_names_list.append(models2.iloc[ran]['event_all']+'_'+str(ran))

        path_g00 = root + '.g00.gz'
        g00_file = get_gz_array_g00(path_g00, skiprows=0)
        rock_types_id = create_rock_type_dict(g00_file)
        models_types_id_list.append(rock_types_id)

        z = z + 1

    return models_list, models_names_list, models_types_id_list

'/home/nikopol/Projects/mm_workflow'